# Integration on level set domains
* simple example: calculate area of the unit circle using level set representations
* background domain: $[-1.5, 1.5]^2 \subset \mathbb{R}^2$.

Let $\phi$ be a continuous level set function. We recall:
$$
  \Omega_{-} := \{ \phi < 0 \}, \quad
  \Omega_{+} := \{ \phi > 0 \}, \quad
  \Gamma := \{ \phi = 0 \}.
$$

In [ ]:
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# ngsolve
from ngsolve import *
# basic xfem functionality
from xfem import *
# for isoparametric mapping
from xfem.lsetcurv import *
# visualisation
from ngsolve.webgui import *
DrawDC = MakeDiscontinuousDraw(Draw)
# the contant pi
from math import pi

We generate the background mesh of the domain and use a simplicial triangulation

In [ ]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5], [1.5,1.5], bc=1)
mesh = Mesh(square.GenerateMesh(maxh=0.8, quad_dominated=False))
Draw(mesh)

On the background mesh we define the level set function. In this example we choose a levelset function which measures the distance to the origin and substracts a constant $r$, resulting in an circle of radius $r$ as $\Omega_{-}$:

In [ ]:
r = 1
levelset = sqrt(x**2+y**2) - r
DrawDC(levelset, -3.5, 2.5, mesh, 'levelset')

* Area of the unit circle:
$$
A = \pi = \int_{\Omega_-(\phi)} 1 \, \mathrm{d}x
$$

* The ngsxfem integration on implicitly defined geometries is only able to handle functions $\phi$ which are in $\mathcal{P}^1(T)$ for all $T$ of the triangulation.

* Therefore we replace the levelset function $\phi$ with an approximation $\phi^\ast \in \mathcal{P}^1(T), T \subset \Omega$. 

In [ ]:
V = H1(mesh,order=1)
lset_approx = GridFunction(V)
InterpolateToP1(levelset, lset_approx)
DrawDC(lset_approx, -3.5, 2.5, mesh, 'levelset_p1')

Notice that this replacement $\phi \mapsto \phi^\ast$ will introduce a geometry error of order $h^2$. We will investigate this numerically later on in this notebook.

As a next step we need to define the integrand:

In [ ]:
f = CoefficientFunction(1)

To call the `Integrate` function -- additionally to the arguments known from standard NGSolve -- we need to pass the "levelset_domain" dictionary argument which has the following entries:
* "levelset": level set function which describes the geometry (best choice: P1-GridFunction)
* "domain_type": decision on which part of the geometry the integration should take place (`NEG`/`POS`/`IF`)

In [ ]:
order = 2
integral = Integrate(levelset_domain = { "levelset" : lset_approx, "domain_type" : NEG},
                             cf=f, mesh=mesh, order = order)
error = abs(integral - pi)
print("Result of the integration: ", integral)
print("Error of the integration: ", error)

### Convergence 

To get more accurate results
* save the previous error and execute a refinement 
* repeat the steps from above
* Repeadetly execute the lower block to add more refinement steps.

In [ ]:
errors = []

def AppendResultAndPostProcess(integral):
    error = abs(integral - pi)
    print("Result of the integration: ", integral)
    print("Error of the integration: ", error)

    errors.append(error)
    eoc = [log(errors[i+1]/errors[i])/log(0.5) for i in range(len(errors)-1)]

    print("Collected errors:", errors)
    print("experimental order of convergence (L2):", eoc)    

AppendResultAndPostProcess(integral)

In [ ]:
# refine cut elements only:
RefineAtLevelSet(gf=lset_approx)
mesh.Refine()

V.Update()
lset_approx.Update()

InterpolateToP1(levelset,lset_approx)
Draw(lset_approx, mesh, "lsetp1", min=0, max=0.0)

integral = Integrate(levelset_domain = { "levelset" : lset_approx, "domain_type" : NEG},
                     cf = f, mesh = mesh, order = order)
AppendResultAndPostProcess(integral)

We observe only a second order convergence which result from the geometry approximation.

## Higher order geometry approximation with an isoparametric mapping
In order to get higher order convergence we can use the isoparametric mapping functionality of xfem.

We apply a mesh transformation technique in the spirit of isoparametric finite elements:
![title](graphics/lsetcurv.jpg)

### Video of the mesh transformation

In [ ]:
%%html
<iframe width=100% height=600px src="https://www.youtube.com/embed/Mst_LvfgPCg?rel=0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

To compute the corresponding mapping we use the LevelSetMeshAdaptation class

In [ ]:
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2, threshold=1000, discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
Draw(deformation,mesh,"deformation")

We can observe the geometrical improvement in the following sequence:

In [ ]:
scene1 = Draw(deformation, mesh, "deformation", autoscale=False, min=0.0, max=0.028)

In [ ]:
scene2 = DrawDC(lsetmeshadap.lset_p1, -3.5, 2.5, mesh, "lsetp1_ho", deformation=deformation)

In [ ]:
N=1000

deformation.vec.data = 1.0/N*deformation.vec
for i in range (1, N+1):
    deformation.vec.data = (i+1)/i * deformation.vec
    scene1.Redraw()
    scene2.Redraw()

## Convergence study with isoparametric mapping applied
### error tables:

In [ ]:
order = 2
refinements = 7
mesh = Mesh (square.GenerateMesh(maxh=0.8, quad_dominated=False))

levelset = sqrt(x*x+y*y)-1
referencevals = { POS : 9-pi, NEG : pi, IF : 2*pi }

lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.2, discontinuous_qn=True)
lsetp1 = lsetmeshadap.lset_p1
errors_uncurved = dict()
errors_curved = dict()
eoc_uncurved = dict()
eoc_curved = dict()

for key in [NEG,POS,IF]:
    errors_curved[key] = []
    errors_uncurved[key] = []
    eoc_curved[key] = []
    eoc_uncurved[key] = []

### refinements:

In [ ]:
f = CoefficientFunction (1.0)

for reflevel in range(refinements):
    if(reflevel > 0):
        mesh.Refine()

    for key in [NEG,POS,IF]:
        # Applying the mesh deformation
        deformation = lsetmeshadap.CalcDeformation(levelset)

        integrals_uncurved = Integrate(levelset_domain = { "levelset" : lsetp1, "domain_type" : key},
                                        cf=f, mesh=mesh, order = order)

        mesh.SetDeformation(deformation)
        integrals_curved = Integrate(levelset_domain = { "levelset" : lsetp1, "domain_type" : key},
                                       cf=f, mesh=mesh, order = order)
        # Unapply the mesh deformation (for refinement)
        mesh.UnsetDeformation()

        errors_curved[key].append(abs(integrals_curved - referencevals[key]))
        errors_uncurved[key].append(abs(integrals_uncurved - referencevals[key]))
    # refine cut elements:
    RefineAtLevelSet(gf=lsetmeshadap.lset_p1)



### Convergence rates:

In [ ]:
for key in [NEG,POS,IF]:
    eoc_curved[key] = [log(a/b)/log(2) for (a,b) in zip (errors_curved[key][0:-1],errors_curved[key][1:]) ]
    eoc_uncurved[key] = [log(a/b)/log(2) for (a,b) in zip (errors_uncurved[key][0:-1],errors_uncurved[key][1:]) ]

print("errors (  curved):  \n{}\n".format(  errors_curved))
print("   eoc (  curved):  \n{}\n".format(     eoc_curved))

* The integration on level set domains (and the mapping) can be extended to `SymbolicBFI` / `SymbolicLFI` ...